# Importo todas las dependencias

In [165]:
import pandas as pd #estadistica
import re           #regular expressions
from unicodedata import normalize       # para quitar tildes
import glob          # para sacar lista de archivos rapidamente

# Creo diccionario entre IPC y diccionario ADR

Crearé un diccionario entre IPC y el diccionario de ADR, para esto abro los datasets de los nombres del IPC y del diccionario para ADR. Este diccionario me servirá para encontrar cuales cultivos se miran en el IPC

In [166]:
# Abrir dictionary ADR
dictionary_ADR = pd.read_csv("data/cadenas_productivas_ADR.csv", sep=';', engine='python')
# Abrir dictionary IPC
IPC_dictionary = pd.read_excel("Data/Ej-reconstruccion-2009-2018-nomenclatura-COICOP-y-var-anuales.xlsx", sheet_name=4, skiprows=[0,1])
# Quitar columnas y filas que no sirven
IPC_dictionary.drop(IPC_dictionary.tail(6).index,inplace=True)
IPC_dictionary.drop(IPC_dictionary.columns.difference(['Código COICOP','Nombre COICOP','Código gasto básico IPC base 2008','Nombre del gasto básico IPC base 2008']), 1, inplace=True)
# Pasar nombres a mayuscula para facilitar comparación
IPC_dictionary['Nombre COICOP'] = IPC_dictionary['Nombre COICOP'].str.upper()
IPC_dictionary['Nombre del gasto básico IPC base 2008'] = IPC_dictionary['Nombre del gasto básico IPC base 2008'].str.upper()


Creo diccionario entre ADR y IPC

In [167]:
# Comparo los datos solo con el 3 nivel del diccionario ADR porque es el que tiene todos los cultivos
dictionary_ADR = dictionary_ADR[dictionary_ADR["NIVEL"]==3]
df = IPC_dictionary
# Quito tildes
dictionary_ADR["CADENA_PRODUCTIVA_ADR"] = dictionary_ADR["CADENA_PRODUCTIVA_ADR"].apply(lambda x: re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", x), 0, re.I))
df["Nombre del gasto básico IPC base 2008"]= df["Nombre del gasto básico IPC base 2008"].apply(lambda x: re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", x), 0, re.I))

# Comparo solo los datos que se emplean desde el 2018(los otros son mas pocos)
IPCs_cadenas = IPC_dictionary["Nombre del gasto básico IPC base 2008"]
cadenas_dictionary = dictionary_ADR["CADENA_PRODUCTIVA_ADR"].tolist()

# dataframe de comparación
df = IPC_dictionary
# Paso datos de las cadenas a lista y hare una comparación por cada cadena de ADR
for i in range(len(cadenas_dictionary)):
    row = cadenas_dictionary[i]
    # Comparo cada registro del diccionario de ADR con los registros del IPC, además se debe de tener en cuenta que deben ser palabras sueltas
    df.loc[((df['Nombre del gasto básico IPC base 2008'].str.contains(rf"\b({row})\b"))|(df['Nombre del gasto básico IPC base 2008'].str.contains(rf"\b({row}+S)\b")))&(~df['Nombre del gasto básico IPC base 2008'].str.contains("BEBIDA"))&(~df['Nombre del gasto básico IPC base 2008'].str.contains("REFRESCO"))&(~df['Nombre del gasto básico IPC base 2008'].str.contains("HARINA"))&(~df['Nombre del gasto básico IPC base 2008'].str.contains("PRODUCTO"))&(~df['Nombre del gasto básico IPC base 2008'].str.contains("PIEZA"))&(~df['Nombre del gasto básico IPC base 2008'].str.contains("FRITURA")), 'Cultivo'] = row
#quito nans (los que no se encontraron)
df=df.dropna()
df


,Código COICOP,Nombre COICOP,Código gasto básico IPC base 2008,Nombre del gasto básico IPC base 2008,Cultivo
0,111.0,PAN Y CEREALES ...,1110100.0,ARROZ,ARROZ
16,114.0,"LECHE, QUESO Y HUEVOS ...",1720300.0,OTROS DERIVADOS LACTEOS,LACTEO
19,116.0,FRUTAS,1410100.0,NARANJAS,NARANJA
20,116.0,FRUTAS,1410200.0,BANANOS,BANANO
21,116.0,FRUTAS,1410300.0,TOMATE DE ARBOL,TOMATE DE ARBOL
22,116.0,FRUTAS,1410400.0,MORAS,MORA
24,117.0,LEGUMBRES-HORTALIZAS,1210100.0,PAPA,PAPA
25,117.0,LEGUMBRES-HORTALIZAS,1210200.0,YUCA,YUCA
27,117.0,LEGUMBRES-HORTALIZAS,1220100.0,PLATANO,PLATANO
28,117.0,LEGUMBRES-HORTALIZAS,1310100.0,CEBOLLA,CEBOLLA


# Usando los datos entre 2009 y 2018

In [168]:
# leer datos 
datos2018 = pd.read_excel("Data/Ej-reconstruccion-2009-2018-nomenclatura-COICOP-y-var-anuales.xlsx", sheet_name=6, skiprows=[0])
#quitar ultimas filas que no tienen datos
datos2018.drop(datos2018.tail(8).index,inplace=True)
# paso a mayuscula para facilitar comparación
datos2018['Nombre COICOP'] = datos2018['Nombre COICOP'].str.upper()
# usando el diccionario anterior agrego datos que están
datos2018_mod = pd.merge(datos2018, df, on="Nombre COICOP")
# Quito columnas que no me sirven para nada
datos2018_mod.drop(IPC_dictionary.columns.difference(['Año', 'Mes', 'Jerarquía', 'Código COICOP', 'Nombre COICOP',
       'Peso IPC base 2008%*', 'Índice base diciembre de 2008',
       'Índice base diciembre de 2018', 'Variación Mensual',
       'Variación Año Corrido', 'Variación Anual',
       'Código gasto básico IPC base 2008',
       'Nombre del gasto básico IPC base 2008', 'Cultivo']), 1, inplace=True)
datos2018_mod.head(2)

,Año,Mes,Jerarquía,Código COICOP_x,Nombre COICOP,Peso IPC base 2008%*,Índice base diciembre de 2008,Índice base diciembre de 2018,Variación Mensual,Variación Año Corrido,Variación Anual,Unnamed: 11,Unnamed: 12,Código COICOP_y,Código gasto básico IPC base 2008,Nombre del gasto básico IPC base 2008,Cultivo
0,2009,Enero,CLASE,1110000.0,PAN Y CEREALES ...,3.75,100.51,85.03,0.51,0.51,NaN,NaN,NaN,111.0,1110100.0,ARROZ,ARROZ
1,2009,Febrero,CLASE,1110000.0,PAN Y CEREALES ...,3.75,100.07,84.66,-0.44,0.07,NaN,NaN,NaN,111.0,1110100.0,ARROZ,ARROZ


Para pasar los meses a numeros uso la siguiente función:

In [169]:
def normalizaraño(dataframe):
    dataframe.loc[dataframe['Mes'].str.contains("Enero"),'Mes']="1"
    dataframe.loc[dataframe['Mes'].str.contains("Febrero"),'Mes']="2"
    dataframe.loc[dataframe['Mes'].str.contains("Marzo"),'Mes']="3"
    dataframe.loc[dataframe['Mes'].str.contains("Abril"),'Mes']="4"
    dataframe.loc[dataframe['Mes'].str.contains("Mayo"),'Mes']="5"
    dataframe.loc[dataframe['Mes'].str.contains("Junio"),'Mes']="6"
    dataframe.loc[dataframe['Mes'].str.contains("Julio"),'Mes']="7"
    dataframe.loc[dataframe['Mes'].str.contains("Agosto"),'Mes']="8"
    dataframe.loc[dataframe['Mes'].str.contains("Septiembre"),'Mes']="9"
    dataframe.loc[dataframe['Mes'].str.contains("Octubre"),'Mes']="10"
    dataframe.loc[dataframe['Mes'].str.contains("Noviembre"),'Mes']="11"
    dataframe.loc[dataframe['Mes'].str.contains("Diciembre"),'Mes']="12"
    return(dataframe)

Creo un archivo de entrega para sacar la información más importante hasta el 2018

In [170]:
entrega2018 = datos2018_mod
# convierto meses a numeros
entrega2018 = normalizaraño(entrega2018)
# creo columnas importantes
entrega2018['Índice']=entrega2018['Índice base diciembre de 2018']
entrega2018['Subclase']=entrega2018['Nombre del gasto básico IPC base 2008']
# quito columnas no importantes
entrega2018.drop(entrega2018.columns.difference(['Año', 'Mes', 'Índice','Cultivo','Subclase']), 1, inplace=True)
# archivo de entrega
entrega2018=entrega2018.sort_values(["Año", "Mes"], ascending = (False, False))
entrega2018.to_csv (r"data/IPC_cultivo_hasta_2018.csv", index = False, header=True)


# Usando los datos desde el 2019

Como toca abrir archivo por archivo, creo funciones que me permitan facilitar las cosas

In [171]:
# Cada archivo tiene los datos en una hoja diferente... con esto detecto en cual hoja están los datos que necesito
def getsheetnumber(file):
    xl = pd.ExcelFile(file)
    for i in range(len(xl.sheet_names)):
        sheet = xl.sheet_names[i]
        df = pd.read_excel(xl, sheet,skiprows=[0,1])
        if "IPC. Variaciones y contribuciones mensual, año corrido y anual según subclases" in df.columns:
            return(sheet)

# Esta función hace lo mismo que hice con la creación del diccionario, pero esta vez para cada archivo que abro, pues la DIAN cada mes quita y pone nuevos productos (en los últimos años)
def insertcultivo(file,sheet):
    # abro archivo IPC
    IPC_dictionary = pd.read_excel(file, sheet_name=sheet, skiprows=range(0,7))
    # Quitar columnas que no sirven
    IPC_dictionary.drop(IPC_dictionary.tail(6).index,inplace=True)
    IPC_dictionary.drop(IPC_dictionary.columns.difference(['Subclase','Índice']), 1, inplace=True)
    IPC_dictionary=IPC_dictionary.dropna()
    # Agrego columna de mes de acuerdo con el nombre del archivo. los de la dian desde el 2020 no ponen esta columna
    if "ene" in file:
        IPC_dictionary["Mes"]=1
    elif "feb" in file:
        IPC_dictionary["Mes"]=2
    elif "mar" in file:
        IPC_dictionary["Mes"]=3
    elif "abr" in file:
        IPC_dictionary["Mes"]=4
    elif "may" in file:
        IPC_dictionary["Mes"]=5
    elif "jun" in file:
        IPC_dictionary["Mes"]=6
    elif "jul" in file:
        IPC_dictionary["Mes"]=7
    elif "ago" in file:
        IPC_dictionary["Mes"]=8
    elif "sep" in file:
        IPC_dictionary["Mes"]=9
    elif "oct" in file:
        IPC_dictionary["Mes"]=10
    elif "nov" in file:
        IPC_dictionary["Mes"]=11
    elif "dic" in file:
        IPC_dictionary["Mes"]=12
    # Agrego columna de año deacuerdo con el nombre del archivo
    if str(20) in file:
        IPC_dictionary["Año"]=2020
    elif str(19) in file:
        IPC_dictionary["Año"]=2019
    # Pasar nombres a mayuscula para facilitar comparación
    IPC_dictionary['Subclase'] = IPC_dictionary['Subclase'].str.upper()
    #quitar tildes
    IPC_dictionary["Subclase"]= IPC_dictionary["Subclase"].apply(lambda x: re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", x), 0, re.I))
    # Comparo solo los datos que se emplean desde el 2018
    IPCs_cadenas = IPC_dictionary["Subclase"]
    cadenas_dictionary = dictionary_ADR["CADENA_PRODUCTIVA_ADR"].tolist()
    # dataframe de comparación
    df = IPC_dictionary
    # Paso datos de las cadenas a lista y hare una comparación por cada cadena de ADR
    for i in range(len(cadenas_dictionary)):
        # cultivo que voy a comparar
        row = cadenas_dictionary[i]
        # comparo por cultivo, sin embargo quito cosas que no son de cultivos, como refrescos, piezas, frituras, etc
        df.loc[((df['Subclase'].str.contains(rf"\b({row})\b"))|(df['Subclase'].str.contains(rf"\b({row}+S)\b")))&(~df['Subclase'].str.contains("BEBIDA"))&(~df['Subclase'].str.contains("REFRESCO"))&(~df['Subclase'].str.contains("HARINA"))&(~df['Subclase'].str.contains("PRODUCTO"))&(~df['Subclase'].str.contains("PIEZA"))&(~df['Subclase'].str.contains("FRITURA")), 'Cultivo'] = row
    # quito cosas que no se encontraron
    df=df.dropna()
    return(df)

Ejemplo de como funciona para los datos de septiembre 2019

In [172]:
file = "Data/anexo_ipc_sep19.xlsx"
sheet = getsheetnumber(file)
dataframe = insertcultivo(file,sheet)
dataframe.head()

,Subclase,Índice,Mes,Año,Cultivo
1,ARROZ,108.86,9,2019,ARROZ
3,AVENA Y SUS DERIVADOS,96.04,9,2019,AVENA
4,TRIGO Y SUS DERIVADOS,102.33,9,2019,TRIGO
5,MAIZ Y SUS DERIVADOS,105.83,9,2019,MAIZ
23,NARANJAS,117.38,9,2019,NARANJA


Ahora si para cada archivo despues del 2018

In [173]:
# saco lista de archivos
files = glob.glob('Data/anexo_ipc*.xlsx')
Data = []
# por cada archivo que tengo
for i in range(len(files)):
    # si es el primer archivo, creo un dataframe inicial 2019
    if i == 0:
        sheet = getsheetnumber(files[i])
        data2019 = insertcultivo(files[i],sheet)
        print("longitud inicial",len(data2019))
    else:
# despues del primer ddataframe creado, lo unico que hago es ir agregando datos a este dataframe
        sheet = getsheetnumber(files[i])
        # dataframe secundario
        dataframe0 = insertcultivo(files[i],sheet)
        print("longitud secundario",len(dataframe0))
        # dataframe que cree adjuntandole además el dataframe secundario
        data2019=data2019.append(dataframe0)
        print("longitud total",len(data2019))

longitud inicial 16
longitud secundario 16
longitud total 32
longitud secundario 16
longitud total 48
longitud secundario 16
longitud total 64
longitud secundario 16
longitud total 80
longitud secundario 16
longitud total 96
longitud secundario 16
longitud total 112
longitud secundario 16
longitud total 128
longitud secundario 16
longitud total 144
longitud secundario 16
longitud total 160
longitud secundario 16
longitud total 176
longitud secundario 16
longitud total 192
longitud secundario 16
longitud total 208
longitud secundario 16
longitud total 224
longitud secundario 16
longitud total 240
longitud secundario 16
longitud total 256
longitud secundario 16
longitud total 272


Organizo y exporto el dataframe en csv

In [174]:
data2019 = data2019.sort_values(["Año", "Mes"], ascending = (False, False))
entrega2019 = data2019
entrega2019.to_csv (r"data/IPC_cultivo_desde_2019.csv", index = False, header=True)

# Exporto dataframe con todo

In [175]:
# agrego dataframes de 2018 y 2019
entrega = entrega2018.append(entrega2019)
# los organizo
entrega=entrega.sort_values(["Año", "Mes"], ascending = (False, False))
# Los paso a csv
entrega.to_csv (r"data/IPC_cultivo.csv", index = False, header=True)